In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import ipyvuetify as v

## CubeViz Example

In [ ]:
url = 'https://dr15.sdss.org/sas/dr15/manga/spectro/redux/v2_4_3/7495/stack/manga-7495-12704-LOGCUBE.fits.gz'
filename = url.split('/')[-1]
if filename.endswith('.gz'):
    ungz_filename = filename[:-3]

In [ ]:
import os
from urllib.request import urlretrieve

if os.path.isfile(ungz_filename):
    print("Ungzipped version of file", filename, 'present, using that')
    filename = ungz_filename
elif os.path.isfile(filename):
    print("File", filename, 'already downloaded')
else:
    try:
        import tqdm
        with tqdm.tqdm_notebook(unit='B', unit_scale=True) as t:
            def reporthook(blocknum, readsize, totalsize):
                if blocknum %100 == 0:
                    t.total = totalsize
                    t.n = blocknum*readsize
                    t.display()
            urlretrieve(url, filename, reporthook)
            t.n =  t.total
    except ImportError:
        urlretrieve(url, filename)
    print("Downloaded", filename)

### Create the VizApp for state information

In [ ]:
from jdaviz.vizcomponents.applications import cubeviz
from jdaviz.vizcomponents.vizapp import VizApp
vizapp = VizApp()

### Create CubeViz and Show

In [ ]:
# it turns out this file has incorrect units in its wcs.  So we fix up the WCS manually... which we can do in the notebook for this particular dataset!
def process_data(cv):
    w = cv._vizapp._glue_app.data_collection[0].coords.wcs
    w.wcs.crval[-1]*=1e10
    w.wcs.cd[2, 2]*=1e10
    repr(w.wcs)
    w.world_axis_units[-1] = 'angstrom'

In [ ]:
c = cubeviz.CubeViz(filename, vizapp, process_data=process_data)

In [ ]:
c.show()

We can manually set the viewer bounds programatically if desired:

In [ ]:
s = c.pviewer._v1d.state
s.y_min, s.y_max  = 0, .3
s.x_min, s.x_max = 6000, 7500

Or we can even do scientific analysis tasks on the selected spectrum:

In [ ]:
from astropy import units as u
from specutils import analysis, SpectralRegion, Spectrum1D

# get the selection
spec = c.pviewer.get_spectrum1D()

analysis.centroid(spec, SpectralRegion(6760*u.angstrom, 6790*u.angstrom))

This can even be interactive - e.g., this will compute the centroid of the line selected in the profile viewer (although it will be empty if there is no selection):

In [ ]:
pv = c.pviewer
analysis.centroid(pv.get_spectrum1D(), pv.get_region())